In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
import houseprice_functions as hpf
from sklearn.linear_model import ElasticNetCV, ElasticNet
from sklearn import metrics
from sklearn import impute
from sklearn.model_selection import GridSearchCV
KNN = impute.KNNImputer
KNN = KNN()

In [2]:
cleaned_hp = pd.read_csv('data/cleaned_houseprice.csv')
HousePrices = pd.read_csv('data/train.csv')
cleaned_hp_test = pd.read_csv('data/cleaned_houseprice_test.csv')
y = HousePrices[["Id",'SalePrice']]
x_y = cleaned_hp.merge(y,how = "inner",on = "Id")

In [3]:
train_outliers = hpf.outlier_selecter(cleaned_hp,cleaned_hp,num_sd = 4,min_unique = 20, drop_zeros = True)
test_range = hpf.outlier_selecter(cleaned_hp,cleaned_hp,num_sd = 4,min_unique = 20, drop_zeros = True,for_test = True)

## Train

In [4]:
#replaces outliers with NAN
for col, idx in train_outliers.items():
    cleaned_hp.loc[:,col].iloc[idx] = np.nan
    
columns_ = cleaned_hp.columns
cleaned_hp = KNN.fit_transform(cleaned_hp)
cleaned_hp = pd.DataFrame(cleaned_hp,columns = columns_)
cleaned_hp.head()

C:\Users\Micha\Downloads\Anaconda\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Id,CentralAir,HeatingQC,garage_score,Heating,Electrical,GarageArea,finishedbsmt,GrLivArea,TotalBsmtSF,...,Ext_Stucco,Ext_WdSdng,Ext_WdShing,House_1.5Fin,House_1.5Unf,House_2.5Fin,House_2.5Unf,House_2Story,House_SFoyer,House_SLvl
0,1.0,1.0,5.0,6.0,1.0,5.0,548.0,0.824766,1710.0,856.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2.0,1.0,5.0,6.0,1.0,5.0,460.0,0.774960,1262.0,1262.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,1.0,5.0,6.0,1.0,5.0,608.0,0.528261,1786.0,920.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4.0,1.0,4.0,6.0,1.0,5.0,642.0,0.285714,1717.0,756.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5.0,1.0,5.0,6.0,1.0,5.0,836.0,0.572052,2198.0,1145.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Test

In [5]:
#replaces outliers with NAN
for col,low_up_sd in test_range.items():
    outliers = cleaned_hp_test.index[cleaned_hp_test.loc[:,col].apply(lambda x: (x < low_up_sd[0]) or (x > low_up_sd[1]))].to_list()
    if len(outliers) != 0:
        cleaned_hp_test.loc[:,col].iloc[outliers] = np.nan
        print(col,':',outliers)

columns_ = cleaned_hp_test.columns
cleaned_hp_test = KNN.fit_transform(cleaned_hp_test)
cleaned_hp_test = pd.DataFrame(cleaned_hp_test,columns = columns_)
cleaned_hp_test.head()


GarageArea : [495, 611, 701]
GrLivArea : [728, 1089, 1362]
TotalBsmtSF : [19, 1089]
LotArea : [790, 803]
EnclosedPorch : [1043, 1107]
Total_PorchDeckSF : [69, 283, 1089, 1146, 1222]


C:\Users\Micha\Downloads\Anaconda\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Id,CentralAir,HeatingQC,garage_score,Heating,Electrical,GarageArea,finishedbsmt,GrLivArea,TotalBsmtSF,...,Ext_Plywood,Ext_Stucco,Ext_WdSdng,Ext_WdShing,House_1.5Fin,House_1.5Unf,House_2.5Unf,House_2Story,House_SFoyer,House_SLvl
0,1461.0,1.0,3.0,6.0,1.0,5.0,730.0,0.693878,896.0,882.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1462.0,1.0,3.0,6.0,1.0,5.0,312.0,0.694507,1329.0,1329.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1463.0,1.0,4.0,6.0,1.0,5.0,482.0,0.852371,1629.0,928.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1464.0,1.0,5.0,6.0,1.0,5.0,470.0,0.650108,1604.0,926.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1465.0,1.0,5.0,6.0,1.0,5.0,506.0,0.205469,1280.0,1280.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
#TRAIN X y
y_train = np.log(x_y['SalePrice'])
X_train = cleaned_hp.drop(["Id","Ext_ImStucc", "Ext_Stone","House_2.5Fin"],axis = 1)
#TRAIN X
X_test = cleaned_hp_test.drop(["Id"],axis = 1)

In [7]:
for col in X_train.columns:
    if (X_train[col].nunique() != 2) and (X_train[col].skew() > 0.5):
        print(col)
        print(X_train[col].skew())
        X_train[col] = np.log1p(X_train[col])
        print(X_train[col].skew())
        print('-'*50)

GrLivArea
0.7945580682885606
-0.12900470046311321
--------------------------------------------------
Fireplaces
0.6495651830548841
0.18208270108947197
--------------------------------------------------
LotArea
2.5023009425758294
-0.9136315930988614
--------------------------------------------------
LotShape
1.309985656555955
0.7831035648282987
--------------------------------------------------
LandSlope
4.813682424489448
4.297167061742489
--------------------------------------------------
EnclosedPorch
2.8793402413491145
2.118754318995043
--------------------------------------------------
Total_PorchDeckSF
1.0106992266380932
-0.9071782105229678
--------------------------------------------------


In [8]:
for col in X_test.columns:
    if (X_test[col].nunique() != 2) and (X_test[col].skew() > 0.5):
        print(col)
        print(X_test[col].skew())
        X_test[col] = np.log1p(X_test[col])
        print(X_test[col].skew())
        print('-'*50)

GrLivArea
0.8183697282292276
-0.042539082761763256
--------------------------------------------------
TotalBath
0.6913998484424572
-0.1241776267556393
--------------------------------------------------
Fireplaces
0.8198582704555165
0.29412900401924885
--------------------------------------------------
LotArea
2.552806435590781
-0.9920046340781267
--------------------------------------------------
LotShape
1.2042262704496145
0.7679560899873284
--------------------------------------------------
LandSlope
4.968389880056036
4.64800252535215
--------------------------------------------------
EnclosedPorch
2.951827718959102
1.8226276994027943
--------------------------------------------------
Total_PorchDeckSF
1.010585450619311
-0.9460523587305861
--------------------------------------------------


In [9]:
#Model Training
en_model = ElasticNetCV(l1_ratio=[0.5,0.6,0.65,0.7,0.75],n_alphas = 720, eps=1e-4, fit_intercept=True, 
                        normalize=True, precompute='auto', max_iter=2000, tol=0.0001, cv=10, 
                        copy_X=True, verbose=0, n_jobs=-1, positive=False, random_state=1)

full_model = en_model.fit(X_train, y_train)
y_pred = full_model.predict(X_train)
print("Full Model RMSE:",np.sqrt(metrics.mean_squared_error(y_pred, y_train)))
print("Full model R2:",en_model.score(X_train,y_train))

Full Model RMSE: 0.11463801499634067
Full model R2: 0.9175812610345955


# END TESTING

In [10]:
y_log_values = full_model.predict(X_test)
y_test = np.exp(y_log_values)
results = pd.DataFrame(zip(cleaned_hp_test.Id.astype(int),y_test),columns = ["Id","SalePrice"])
results.head(10)

,Id,SalePrice
0,1461,122756.569353
1,1462,153837.792422
2,1463,168071.221742
3,1464,185819.051005
4,1465,195349.734217
5,1466,157568.690587
6,1467,163702.626457
7,1468,151618.247547
8,1469,183331.924357
9,1470,112775.565841


In [ ]:
test_submission = results.to_csv('data/submission.csv',index = False)